## Shock and awe: 1.19. [Inter-process communication (IPC sockets): synchronous callbacks](https://code.kx.com/q4m3/1_Q_Shock_and_Awe/#119-interprocess-communication-101)

Server side:
- opening a port \p 30210
- \p checking if port is open
- assign the port to a host: It has the same syntax as file handling:
    - `:localhost:30210

Client side:
- open connection on client and assign it to a variable:
    - h:hopen `:localhost:30210
    - calling the h variable / function handles a synchronous request to the server
    - variable h is called an open handle: it holds a function to send and receive requests to / from the server
    - e.g.: h "4*6" sends the string to the server which calculates it and sends it back
- close connection: hclose h

Synchronous means that the client blocks until receives the response from the server

In [ ]:
.simbud.conn: hopen hsym `$"localhost:8060"

In [ ]:
2+3

Send expressions to or call functions on server:
h (`func_name;param1;param2...)

## I/O: 11.6. [Inter-process communication](https://code.kx.com/q4m3/11_IO/#116-interprocess-communication)

- Basic terminology:
    - Client: the process that initiates the communication
    - Server: the process receiving and processing requests.The server process can be on the same machine, same or different network or on the Internet
    Synchronous communication: waits for a result to be returned
    Asynchronous communication: does not wait and no result returned

### 11.6.1 Communication Handle

- A symbolic communication handle (a.k.a network handle) has the general form: `:[serverIdentifier]:port.
- A server identifier can refer to:
    - your localhost `:localhost:5042 (omitted server identifier also refers to localhost: `::5042)
    - a machine on the network:
        - by name: `:aerowing:5042
    - the IP address of a machine: `:198.162.0.2:5042
    - a URL: `:www.myurl.com:5042

### 11.6.2 Operning a connection Handle

- open communication handle on server: `:server:port
- open connection handle on client: h:hopen `:server:port
- send query: h"3+4" / UNSAFE!!
- close connection when communication with the server is finished: hclose h

### 11.6.3 Remote Execution

- connHandle (functionName;arg1;arg2;arg3) / function passed by value to call it on client side
    - UNSAFE to let an arbitrary function be executed on your server!!
- connHandle (`functionName;arg1;arg2;arg3) / function passed by name to call it on server side
    - SAFE: because lets only functions be executed on the server that has already defined on the server.
- format: function application: (list; indices) or (map; keys)

- Q analogue of a remote stored procedure: when the remote function (defined on the server) performs an operation on a table (on the server) and returns the result.
- The difference from SQL stored procedures is that the remote procedure can be any q function on the server, making the full power of q available remotely.

### 11.6.4 Synchronous and Asynchronous Messages

- During a synchronous IPC, upon application of the connection handle, the client process blocks, waiting for a result from the server before proceeding and
- the value returned from the server becomes the __return value of the open handle application__.
- When the message passing is synchronous, the following steps occur in sequence:
    - __The client__
        - sends a message containing the argument(s) of the handle application to the server and
        - and waits for a return message.
    - __The server__
        - receives the message
        - interprets it as the appropriate function application
        - and obtains the result.
        - Then, it sends a message containing the result back to the client.
    - __The client__
        - receives the result and
        - resumes execution from the point it left off.
- When a client sends multiple messages to a server in synchronous message passing,
    - __the next message is not sent until the result of the previous message is received__. Consequently
    - the messages always arrive at the server in the order in which they are sent.
    - Also, __the results from the server arrive back at the client in the order in which the original messages were sent__.

- During asynchronous IPC,
    - the message is sent to the server and execution on the client continues immediately.
    - __There is no return value from the server__!!
    - Use case: initiate a task on the server when you don’t care about the result. E.g.:
        - initiate a long running operation or
        - send a message that the server will route to other processes.

In [ ]:
neg[h] (`functionOnServer;arg1;arg2) / asynchronous syntax on client side

- Important!!
    - When sending asynchronous messages, __always send an empty “chaser” message immediately before applying hclose__ to the open handle.
    - If you do not do this, buffered messages may not be sent when the connection is closed.

- Because a q session is single threaded by default, the server will process messages in the order in which they are received.
- However, in asynchronous messaging there is no guarantee that the messages arrive at the server in the order in which they are sent.

## Shock and awe: 1.20. [Example: Asynchronous callbacks](https://code.kx.com/q4m3/1_Q_Shock_and_Awe/#120-example-asynchronous-callbacks)

- in case of anychronous callbacks the client does not block while it waits for the server's response: the application of the open handle returns immediately
- there are no built-in callbacks, but we can create our own
- calling the the "neg h" function handles an asynchronous request to the server
    - (neg h) (`remote_function;param1;param2)
    - (neg .z.w): asynchronously calls back the sender (.z.w: "who is?")

In [ ]:
rsvp:{[arg;cb] show arg; (neg .z.w) (cb; 43); show `done;} / function to call on server.
 / (neg .z.w)(callback_name;callback_param) is the real callback function
echo:{show x} / function to call back on client
(neg h)(`rsvp;"waiting for callback";`echo)

### 11.6.5 Processing messages

- During evaluation of a call from the client, the communication handle of the client process is available in the system variable __.z.w__ ( “who” called).
    - For an asynchronous call, this can be used to send messages back to the client during the function application on the server.
    - When performing asynchronous messaging, always use neg[.z.w] to ensure that all messages are asynchronous. Otherwise you will get a deadlock as each process waits for the other.

In [ ]:
f:{show "Received ",string x; neg[.z.w] (`mycallback; x+1)} / asynchronous callback from server to client

In [ ]:
/ callback function on client side which is triggered by the message sent by the server when it finished evaluating the call
mycallback:{show "Returned ",string x;}

- Both the client and the server have connection handles when a connection between them is opened. However, these handles are assigned independently and their int values are not equal in general.

- You can override the default behavior of message processing in q by assigning your own handler(s) to the appropriate system variables.
- .z.pg (process get, IPC synchronous messages): assigning a function to .z.pg traps and processes synchronous messages on server side.
    - This makes any connHandle"expression" type messages evaluated by the .pg function
- .z.ps (process set, IPC asynchronous messages): assigning a function to .z.ps traps and processes asynchronous messages on server side.
    - This makes any neg[h] "expression" type messages evaluated by the .ps function.

In [ ]:
.z.pg:{$[-11h=type first x; .[value first x; 1_x; ::]; `unsupported]} / on server
h (`sq; 5) / 25 on client
h (`sq; `5) / "type"
h "6*7" / `unsupported
h ({x*y};6;7) / `unsupported

- Useful system variables to handle IPC communication.
- Define functions to be executed when:
    - the connection opens: .z.po
    - the connection closes: .z.pc
- Their sole parameter is the connection handle of the client process.
- Use case: tracking connections of client processes to allow them to register callbacks with the servers.

- An awesome example for creating registered callback functions from clients on server side
https://code.kx.com/q4m3/11_IO/#1165-processing-messages

### 11.6.6. [Remote queries](https://code.kx.com/q4m3/11_IO/#1166-remote-queries)

[Implementation of a remote query](https://code.kx.com/q4m3/11_IO/#1166-remote-queries)

## 11.7. HTTP and Websockets

### 11.7.1. HTTP connections

In [ ]:
 / open a port and create an socket handle
\p 8889
`:localhost:8889

In [ ]:
 / rewrite the default GET request handler namespace .z.ph
.z.ph:{show x 0; show x 1; ; string value 1_x 0}

## Shock and awe: 1.21. [Websockets 101](https://code.kx.com/q4m3/1_Q_Shock_and_Awe/#121-websockets-101)

- [WebSockets](https://en.wikipedia.org/wiki/WebSocket) can be used to put a browser front end on traditional applications, replacing both the web server and proprietary GUI packages
- The key idea of WebSockets:
    - the client makes an initial HTTP request to upgrade the protocol.
    - If the request is accepted, subsequent communication occurs over TCP/IP sockets protocol.
    - Once the WebSockets connection is made, either the client or server can initiate messaging.

In [ ]:
\p 5042 / open port

/
.z.ws: web socket handler
sets the ws handler to a function that will be called on receipt of each message from the browser
-8!"answer" serializes the answer
\
.z.ws:{0N!-9!x; neg[.z.w] -8!42}

In [ ]:
 / broken yahoo link for drawing a chart (ws101.q)

### 11.7.2. Basic WebSockets

- WebSockets is a network protocol that upgrades an initial HTTP handshake into a TCP/IP socket connection.
- It was initially used to enhance communication capability between browsers and web servers
    - but it can be used for general client-server applications.
- Once the WebSocket connection is established, either the client or server can message the other;
    - in particular, this provides the capability for the server to push data to the client.
- Communication through a websocket in q is limited to async messages.

In [ ]:
\p 5042

In [ ]:
.z.ws:{neg[.z.w]x} // default implementation

In [ ]:
.z.ws:{0N!-9!x; neg[.z.w] -8!42}a

In [ ]:
path:"/home/akincsei/00_datasets/kaggle__price-volume-data-for-all-us-stocks-etfs/csv_data/Stocks/"
ticker_list:(`aapl;`amzn;`baba;`fb;`fox;`googl;`ibm;`intc;`lmt;`lhm;`ms;`msft;`orcl;`tmus)
ticker:string ticker_list[1]
ext:".us.csv"
pattern:"DFFFFII"
pth_fn:hsym `$path,ticker,ext
pth_fn

In [ ]:
rd_csv:{[pth;tck;ext;ptrn] (ptrn;enlist ",") 0: hsym `$pth,tck,ext}

In [ ]:
aapl_table:rd_csv[path;ticker;ext;pattern]

In [ ]:
aapl_table

In [ ]:
amzn_str:read0 hsym `$path,ticker,ext

In [ ]:
.z.ws:{0N!-9!x; neg[.z.w] -8!amzn_str}

### 14.7.3. Pushing data to the browser

- In WebSockets the browser initiates the connection, but once the WebSocket request for protocol upgrade is successful, the browser – i.e., client – and the server are on equal footing:
    - Either side can send messsages to the other.
    - All interaction is async in q websocket implementation.
    - Browsers and q is single-threaded by default -> no race conditions and deadlocks.
    - You have to set-up your own callbacks.
- In this section we demonstrate how the q server can push data to the browser, beginning with the browser script.

In [ ]:
\p 4242

In [ ]:
\p

In [ ]:
 / `:localhost:4242

In [ ]:
answer:42

In [ ]:
.z.po:{`requestor set x; system "t 1000";}

In [ ]:
.z.ts:{neg[requestor] -8!answer;; answer+:1;}

[More on websockets](https://code.kx.com/q/wp/websockets/#what-are-websockets)

In [ ]:
345+4

In [ ]:
# 234+4555